<a href="https://colab.research.google.com/github/LI3ARA/FYIRP/blob/main/Notebooks/25/VQA_RAG/Googl_colab/soft_prompting/downloading_data_and_formatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  SPIQA to LLaVA LoRA Fine-Tuning Pipeline (Colab Ready)

# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# 2. Install required libraries
! datasets huggingface_hub
# !git clone https://github.com/haotian-liu/LLaVA.git
# %cd LLaVA
# !pip install -e .

/bin/bash: line 1: datasets: command not found


In [ ]:
#  3. Download SPIQA dataset from Hugging Face
from huggingface_hub import snapshot_download
snapshot_download(repo_id="google/spiqa", repo_type="dataset", local_dir='/content/spiqa')

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

example.json:   0%|          | 0.00/103 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

SPIQA_testA_Images.zip:   0%|          | 0.00/121M [00:00<?, ?B/s]

SPIQA_train_val_test-A_raw_tex.zip:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

testA_data_viewer.json:   0%|          | 0.00/2.89M [00:00<?, ?B/s]

SPIQA_train.json:   0%|          | 0.00/209M [00:00<?, ?B/s]

SPIQA_train_val_Images.zip:   0%|          | 0.00/32.0G [00:00<?, ?B/s]

SPIQA_val.json:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

'/content/spiqa'

In [ ]:
from huggingface_hub import snapshot_download

# Download the entire dataset to '/content/spiqa'
snapshot_download(
    repo_id="google/spiqa",
    repo_type="dataset",
    local_dir='/content/spiqa',
    allow_patterns="train_val/SPIQA_train_val_Images.zip"  # Only download this file
)

In [ ]:
# 4. Unzip relevant data
import zipfile, os

zip_path = "/content/spiqa/train_val/SPIQA_train_val_Images.zip"
extract_path = "/content/spiqa_images"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:

# 5. Load and convert SPIQA_train.json to LLaVA format
import json
import os

with open("/content/spiqa/train_val/SPIQA_train.json", "r") as f:
    spiqa_data = json.load(f)


In [ ]:
spiqa_data['2307.11618v1'].keys()

dict_keys(['paper_id', 'all_figures', 'qa'])

In [ ]:
spiqa_data['2307.11618v1']['qa'][0]

{'question': 'Which method performs the best on the DomainNet dataset with a labeling budget of 5k?',
 'answer': 'DiaNA (Ours)',
 'explanation': 'The table shows the accuracy of different methods on the DomainNet dataset with different labeling budgets. DiaNA (Ours) achieves the highest accuracy of 57.8% with a labeling budget of 5k.',
 'reference': '2307.11618v1-Table1-1.png'}

In [ ]:
spiqa_data['2307.11618v1']['all_figures'].keys()

dict_keys(['2307.11618v1-Table1-1.png', '2307.11618v1-Table2-1.png', '2307.11618v1-Figure2-1.png', '2307.11618v1-Table5-1.png', '2307.11618v1-Figure6-1.png', '2307.11618v1-Figure7-1.png', '2307.11618v1-Figure4-1.png', '2307.11618v1-Table3-1.png', '2307.11618v1-Figure3-1.png', '2307.11618v1-Figure5-1.png', '2307.11618v1-Table4-1.png'])

In [ ]:
spiqa_data['2307.11618v1']['all_figures']['2307.11618v1-Table1-1.png']

{'caption': ' Comparison results (Accuracy: %) on DomainNet with 1k, 2k and 5k labeling budgets. “Random” and “CONF” correspond to the classic AL approaches “Random Sampling” and “Least-Confidence Sampling”.',
 'content_type': 'table',
 'figure_type': 'table'}

In [ ]:
# import json
# import os

# # Load SPIQA data (update path as needed)
# with open("/content/spiqa/train_val/SPIQA_train.json", "r") as f:
#     spiqa_data = json.load(f)

converted = []
image_base_path = "/content/spiqa_images/SPIQA_train_val_Images"  # Where the images were extracted

# Iterate through each paper
for paper_id, paper in spiqa_data.items():
    figures = paper.get("all_figures", {})
    qa_pairs = paper.get("qa", [])

    # print("Nu of qa pairs:", (qa_pairs))

    for qa in qa_pairs:
        ref_img = qa.get("reference")
        question = qa.get("question")
        answer = qa.get("answer")

        if not ref_img or ref_img not in figures:
            continue

        image_path = os.path.join(image_base_path, paper_id, ref_img)
        if not os.path.exists(image_path):
          # print('path does not exist', image_path)
          continue

        # Add to LLaVA-style format
        converted.append({
            "image": image_path,
            "conversations": [
                {
                    "from": "human",
                    "value": f"<image>\n{question}"
                },
                {
                    "from": "system",
                    "value": answer
                }
            ]
        })


In [ ]:
import os

def find_file(root_dir, filename):
  for dirpath, dirnames, filenames in os.walk(root_dir):
    if filename in filenames:
      return os.path.join(dirpath, filename)
  return None

path = find_file("/content", "2310.15484v1-Table5-1.png")

In [ ]:
path

'/content/spiqa_images/SPIQA_train_val_Images/2310.15484v1/2310.15484v1-Table5-1.png'

In [ ]:
len(converted)

262524

In [ ]:
output_path = "/content/spiqa/train_val/SPIQA_train_converted.json"
with open(output_path, "w") as f:
    json.dump(converted, f, indent=2)

print(f" Converted dataset saved to: {output_path}")

 Converted dataset saved to: /content/spiqa/train_val/SPIQA_train_converted.json
